# Analysing Trail Run Results from cycosports

The data is publicly available from https://cycosports.com/2021-jungle-cross-trail-run-april-3rd-4th/ in PDF format. The table data needs to be extracted from the PDF. We will be analysing the results from 4th April.

https://pdftables.com/ was able to reasonably generate Excel/CSV versions of the datset. The CSV version is used. However, this website only offers 25 pages of free conversions. 

An alternative free site which gives a reasonable output is https://www.pdftoexcelconverter.net/.

# 1. Preparing the data

We will aim to transform this data into a tidy data format as defined by Hadley Wickham (https://en.wikipedia.org/wiki/Tidy_data).

First, we import the necessary libraries and load the DataFrame.

In [ ]:
import pandas as pd 
import numpy as np

df = pd.read_csv('results.csv', skiprows=1)

Then, we standardise the format of the column names.

In [ ]:
df = df.rename(columns = {"Pl":"category_rank", "overall":"event_rank", "1stLap":"lap_1", "2ndLap":"lap_2"})
df = df.rename(str.lower, axis="columns")

In [ ]:
df.head()

In [ ]:
df.shape

## Removing the rows with no usable data

* The data contains some rows with the string "Jungle Cross 2021 Trail Run Series Race 2". 
* There are also repeated header rows (with the values "Pl", "overall", "Name", etc.) within the data. This is due to the PDF repeating them for each page. 
* Also, the rows containing "DNS" and "DNF" (in the "Pl" and "overall" data columns) do not have timing information associated with them. 

In [ ]:
df[50:55]

In [ ]:
df.tail()

We will remove those rows from the DataFrame.

In [ ]:
df = df[~df["category_rank"].str.contains("Jungle Cross 2021", na=False, regex=False)]
df = df[~df["name"].str.contains("Jungle Cross 2021", na=False, regex=False)]
df = df[~df["time"].str.contains("Time", na=False, regex=False)]
df = df[~df["category_rank"].str.contains("DNS|DNF", na=False)]

Notes: 

The inversion operator (`~`) is used to return rows not containing the terms. Alternatively, the following can be used:

`df = df[df["column"].str.contains("substring", na=False)==False]`

`regex=False` should not be used if there are regex expressions such as `"DNS|DNF"`.

`na=False` must be used, as the string methods cannot work where values are not a string (e.g. NaN). 
Running the code without `na=False` will result in (for the given column) dropping rows with 
not only the specifed substring, but also those with NaN values.

If we want to verify which rows are being dropped, the code can be run on the original DataFrame but without the inversion, 
to get back a DataFrame with only those rows.

In [ ]:
df[50:55]

In [ ]:
df.tail()

In [ ]:
df.shape

## Dealing with the event and gender data in the "category_rank" column

The DataFrame has additional information in the "category_rank" column. The items are: 
* race event (e.g. "10km - Masters (40+)") 
* gender (Male or Female) 
* age category (e.g. Open, Masters, Under 14) in that order. 

They are in their own header rows with no other information. 

This information has to be split out into their own columns to keep the "category_rank" column clean.

### Creating columns for distance, event and age

We can search for rows containing "km", but with first resetting the index to synchronise it with the actual row numbers:

In [ ]:
df = df.reset_index(drop=True)
df[df["category_rank"].str.contains("km", na=False, regex=False, case=False)]

By running the above code, we can see that there are only 3 race events. Hence, we can use their locations to manually fill a new column via slicing. 

In [ ]:
df["race"] = ""
df.loc[:91, "race"] = "10km - Open - 13+"
df.loc[91:142,"race"] = "10km - Masters - 40+"
df.loc[142:,"race"] = "3km - Adventure Race - 7+"

We then remove the now-redundant header rows. We can also split the information in the new column into more granular columns.

In [ ]:
df = df.drop([0, 91, 142])

df[["distance","event", "age"]] = df["race"].str.split("-",expand=True)
df = df.drop(columns=["race"])

In [ ]:
df.head()

### Creating a column for gender

We can prepare the "gender" column by first making a copy of the "category_rank" column: 

In [ ]:
df["gender"] = df["category_rank"]

However, there are 14 header rows for gender as shown below.

In [ ]:
len(df[df["gender"].str.fullmatch("Male|Female", na=False)])

Hence, we will not use the manual method as shown above, but we will replace all the non-gender values in the column with NaN. This then allows us to forward-fill the values (down the column) from the remaining headers to complete the column.

In [ ]:
df["gender"][~df["gender"].str.fullmatch("Male|Female", na=False)] = np.nan
df["gender"] = df.gender.ffill()

In [ ]:
df.head()

### Creating a column for category

Again, we can create the column by copying the current "category_rank" column. 

In [ ]:
df["category"] = df["category_rank"]

Replacing the rank numbers in the column with NaN values allows us to forward-fill the category information to complete the column.

In [ ]:
df["category"] = df["category"].replace(to_replace='\d+\.\d*', value=np.nan, regex=True)
df["category"] = df["category"].ffill()

In [ ]:
df.head()

Following that, we can strip excess gender information from this column.

In [ ]:
df[["category","sex"]] = df["category"].str.split("-",expand=True)
df = df.drop(columns="sex")

In [ ]:
df.head()

### Removing rows with no time 

At this point, the DataFrame still has the remaining header rows under "category_rank". These rows have no values under the "time" column. 

To clean the data up, we will simply remove all rows with NaN values under "time".

After this, the "category_rank" column should be clean.

In [ ]:
df = df.dropna(subset=["time"])
df = df.reset_index(drop=True)

In [ ]:
df.head()

## Dealing with the bib number

The "name" column has additional information regarding the bib number. It is possible to extract that information into a new column and then remove it from the "name" column. 

In [ ]:
df["bib_number"] = df["name"].str.extract("\((\d+)\)",expand=True)
df["name"] = df["name"].str.split("(", expand=True)

There are some entries without a bib number, so we will fill those with the string "None". 

In [ ]:
df["bib_number"] = df['bib_number'].fillna("None")

In [ ]:
df.head()

## Removing NaN values from the "club" column

For consistency, we will do the same for NaN values in the "club" column as we did for the "bib_number" column.

In [ ]:
df['club'] = df['club'].fillna("None")
df.head()

Now, we have values in all cells of the DataFrame.

In [ ]:
df.isna().sum().sum()

## Properly formatting the duration-based columns

The "lap_1", "lap_2" and "time" columns contain duration information. Analysis requires conversion to the Pandas timedelta object. However, using the columns as they are will throw errors with `pd.to_timedelta`. 

Within the same column, some entries are in `%M:%S.%f`(MM:SS:ff) and some are in `%-H:%M:%S.%f` (H:MM:SS:ff) format. 

In [ ]:
df[60:65]

As long as we change the `%M:%S.%f` entries to `%-H:%M:%S.%f` so that the whole column is consistent, `pd.to_timedelta` will accept it. 

However, we will change both the `%M:%S.%f` and the `%-H:%M:%S.%f` entries in the column to `%H:%M:%S.%f` (HH:MM:SS:ff) as it is more conventional. 

There are three ways we can write functions to update the formats. The functions will add the respective zero digits and semicolons to the strings where applicable. 

### Method 1: Counting the number of ":" in the strings

In [ ]:
def add_hours_zero(column):
    m = column.str.count(':') == 2
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.count(':') == 1
    column = column.mask(m, "00:" + column, axis=0)
    return column

### Method 2: Measuring the length of the string

In [ ]:
def add_hours_zero(column):
    m = column.str.len() == 10
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.len() == 8
    column = column.mask(m, "00:" + column, axis=0)
    return column  

### Method 3: Using regex

In [ ]:
def add_hours_zero(column):
    m = column.str.contains("^\d+:\d+:\d+\.\d+$", na=False)
    column = column.mask(m, "0" + column, axis=0)
    return column

def add_hours(column):
    m = column.str.contains("^\d+:\d+\.\d+$", na=False)
    column = column.mask(m, "00:" + column, axis=0)
    return column

### Applying the functions

Note that `add_hours_zero` must be applied before `add_hours` to work properly on the data.

In [ ]:
df[["lap_1", "lap_2", "time"]] = df[["lap_1", "lap_2", "time"]].apply(add_hours_zero)
df[["lap_1", "lap_2", "time"]] = df[["lap_1", "lap_2", "time"]].apply(add_hours)

In [ ]:
df[60:65]

## Formatting the "start" column to datetime format 

The "start" column contains the race start time of the runner on that day. After formatting it to datetime format, year-month-day placeholders appear in the entries. We can specify the actual date of the event.

In [ ]:
df["start"] = pd.to_datetime(df["start"], format="%H:%M:%S.%f")
df["start"] = df["start"].map(lambda x: x.replace(year=2021, month=4, day=4))

## Casting of types for the rank columns 

The "category_rank" and "event_rank" columns are of dtype: object and appear formatted as a float in the dataframe. We can change them to integers.

In [ ]:
df["category_rank"] = pd.to_numeric(df["category_rank"]).astype(int)
df["event_rank"] = pd.to_numeric(df["event_rank"]).astype(int)

In [ ]:
df.dtypes

## Exporting the cleaned data to .csv 

Now, we can export the data, and re-import it to validate it.

In [ ]:
df.to_csv("results_clean.csv", index=False )
df = pd.read_csv('results_clean.csv')

In [ ]:
df

In [ ]:
df.isna().sum().sum()

--End of Part 1--